<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EA%B3%BC%EC%A0%95/(%EB%85%B8%EC%9D%B4)_%EC%A6%9D%EB%B6%84S%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%86%8C%EB%B6%84%EB%A5%98_%ED%95%99%EC%8A%B5%EA%B2%B0%EA%B3%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 145 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595739 sha256=9ee01d57b9a677c6b7a054759634ecd3901f71645b6255b8e535792061f5a7d6
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.3 MB/s 
     |████████████████████████████████| 769 kB 1.5 MB/s 
     |████████████████████████████████| 3.0 MB 26.4 MB/s 
     |████████████████████████████████| 895 kB 55.1 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-kt5twdap
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pi

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본			   G_inc_so_model_노이2.pt
 산업분류자동화		   G_inc_so_model_노이.pt
'Colab Notebooks'	  'My Drive'
 G_inc_again_so_model.pt   Screenshot_20220401-001411_Chrome.jpg


In [4]:
import pandas as pd
df=pd.read_csv('/mydrive/산업분류자동화/dataset/inc/df_S_inc.csv', sep = ",", encoding = "UTF-8")

In [5]:
df=df.reset_index(drop=True)

## 데이터 전처리

In [6]:
df.head(10)

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000037,S,94.0,949.0,교회 기독교 계통 종교활동 종교 서비스
1,id_0000040,S,94.0,949.0,교회에서 기독교 계통의 종교활동 종교 서비스
2,id_0000058,S,94.0,949.0,교회 기독교 계통 종교활동 종교 서비스
3,id_0000115,S,94.0,949.0,협회에서 게스트하우스 대상으로 관리 및 진흥
4,id_0000135,S,94.0,949.0,교회에서 신자들 대상으로 기독교 계통 종교활동
5,id_0000158,S,94.0,949.0,사업장에서 도시농업 관련 협회 회원관리
6,id_0000166,S,94.0,949.0,교회에서 신자들 대상으로 기독교 단체
7,id_0000187,S,94.0,949.0,사찰 불교 계통 종교활동 종교 서비스
8,id_0000289,S,94.0,949.0,교회에서 기독교 종교활동 종교 서비스 제공
9,id_0000411,S,94.0,949.0,교회 기독교 계통 종교활동 종교 서비스


In [7]:
# 결측값 확인
df.isnull().sum()

AI_id      0
digit_1    0
digit_2    0
digit_3    0
text       0
dtype: int64

In [8]:
df.shape

(163515, 5)

In [9]:
df['digit_2'].value_counts()

94.0    59350
96.0    56210
95.0    47955
Name: digit_2, dtype: int64

In [10]:
df['digit_3'].value_counts().sort_index()

941.0    16120
942.0    21650
949.0    21580
951.0    15080
952.0    12142
953.0    20733
961.0    43040
969.0    13170
Name: digit_3, dtype: int64



#### **추후 과제: 분류가 현재 매우 불균형함 / 정규화를 이용해서 균형을 맞추면 정확도가 올라가는지 확인 필요**


In [11]:
df.loc[(df['digit_3'] == 941), 'digit_3'] = 0  
df.loc[(df['digit_3'] == 942), 'digit_3'] = 1  
df.loc[(df['digit_3'] == 949), 'digit_3'] = 2  
df.loc[(df['digit_3'] == 951), 'digit_3'] = 3  
df.loc[(df['digit_3'] == 952), 'digit_3'] = 4
df.loc[(df['digit_3'] == 953), 'digit_3'] = 5
df.loc[(df['digit_3'] == 961), 'digit_3'] = 6
df.loc[(df['digit_3'] == 969), 'digit_3'] = 7

In [12]:
data_list = []
for q, label in zip(df['text'], df['digit_3'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['교회 기독교 계통 종교활동 종교 서비스', '2.0'], ['교회에서 기독교 계통의 종교활동 종교 서비스', '2.0'], ['교회 기독교 계통 종교활동 종교 서비스', '2.0'], ['협회에서 게스트하우스 대상으로 관리 및 진흥', '2.0'], ['교회에서 신자들 대상으로 기독교 계통 종교활동', '2.0']]


## Train data & test data

In [13]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [18]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(float(i[label_idx])) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [19]:
# Setting parameters
max_len = 64
batch_size = 64 #32,16,8로 감소시키기
warmup_ratio = 0.1
num_epochs = 10 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [20]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [21]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [22]:
data_train[0]

(array([   2, 3312, 6379, 3807, 7119, 7088, 1652, 3312, 6379, 2872, 6559,
           3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(12, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 5)

In [23]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기

In [24]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=8,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [25]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [26]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/2300 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.145434856414795 train acc 0.15625
epoch 1 batch id 201 loss 1.7276030778884888 train acc 0.2583955223880597
epoch 1 batch id 401 loss 0.37709975242614746 train acc 0.5366661471321695
epoch 1 batch id 601 loss 0.12205630540847778 train acc 0.6757487520798668
epoch 1 batch id 801 loss 0.07010018080472946 train acc 0.7478152309612984
epoch 1 batch id 1001 loss 0.08541339635848999 train acc 0.7919112137862138
epoch 1 batch id 1201 loss 0.09683842211961746 train acc 0.8214768942547876
epoch 1 batch id 1401 loss 0.02325146272778511 train acc 0.8426904889364739
epoch 1 batch id 1601 loss 0.1000131219625473 train acc 0.8589455808869456
epoch 1 batch id 1801 loss 0.09309228509664536 train acc 0.8719981954469739
epoch 1 batch id 2001 loss 0.11218338459730148 train acc 0.8822541854072964
epoch 1 batch id 2201 loss 0.11392024159431458 train acc 0.8904688209904589
epoch 1 train acc 0.8940235004025766


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 9 test acc 0.98858642578125


  0%|          | 0/2300 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.024583157151937485 train acc 0.984375
epoch 10 batch id 201 loss 0.0003460294392425567 train acc 0.9948694029850746
epoch 10 batch id 401 loss 0.00020264656632207334 train acc 0.9952072942643392
epoch 10 batch id 601 loss 0.001320856623351574 train acc 0.9957362728785357
epoch 10 batch id 801 loss 0.010931514203548431 train acc 0.9957670099875156
epoch 10 batch id 1001 loss 0.011466655880212784 train acc 0.995801073926074
epoch 10 batch id 1201 loss 0.01849328726530075 train acc 0.9957327227310575
epoch 10 batch id 1401 loss 0.0002952750073745847 train acc 0.9957619557458958
epoch 10 batch id 1601 loss 0.00028489556279964745 train acc 0.9958619612742037
epoch 10 batch id 1801 loss 0.0019852537661790848 train acc 0.9958703498056635
epoch 10 batch id 2001 loss 0.0006546148797497153 train acc 0.9958458270864567
epoch 10 batch id 2201 loss 0.001605809316970408 train acc 0.995790265788278
epoch 10 train acc 0.9957472826086956


  0%|          | 0/256 [00:00<?, ?it/s]

epoch 10 test acc 0.98858642578125


In [27]:
#학습 모델 저장
torch.save(model, 'S_inc_so_model.pt')